# Word game
https://wiki.apterous.org/Letters_game

Contestant chooses 9 letters by selecting either a vowel or a consonant until there is a total of 9 letters. The player can choose the letters in any order, but the selection must include at least 4 consonants and 3 vowels, hence there are only three valid choices in modern Countdown: 3 vowels, 6 consonants; 4 vowels, 5 consonants; and 5 vowels, 4 consonants

Distribution of letters: http://www.thecountdownpage.com/letters.htm

Steps:
1. Create the letter distribution
2. Select the letters. There must be a minimum of 3 vowels and minimum of 4 consonants. Maximum of 5 vowels and 6 consonants. 
3. Create the checker. One to check the user input. Another to find if there's a longer word. 

# Letter Pickers

In [1]:
#https://stackoverflow.com/questions/2570690/python-algorithm-to-randomly-select-a-key-based-on-proportionality-weight
import random

def weighted_pick(dic):
    r = random.uniform(0, sum(dic.values()))
    s = 0.0
    for k, w in dic.items():
        s += w
        if r < s: return k
    return k

In [2]:
def letter_classifier(st):
    vowel_count = 0
    consonant_count = 0
    for i in st:
        if i in 'aeiou':
            vowel_count += 1
        elif i in 'bcdfghjklmnpqrstvwxyz':
            consonant_count += 1
    return vowel_count,consonant_count

In [3]:
def letter_adder(dic):
    letter = weighted_pick(dic)
    dic[letter] -=1
    print(dic)
    return letter

In [4]:
def pick_letters():
    vowels = {"a":15,"e":21,"i":13,"o":13,"u":5}
    consonants = {"b":2,"c":3,"d":6,"f":2,"g":3,"h":2,"j":1,"k":1,"l":5,"m":4,"n":8,"p":4,"q":1,"r":9,"s":9,"t":9,"v":1,"w":1,"x":1,"y":1,"z":1}
    letters = []
    while len(letters)<9:
        choice = input("(V)owel or (C)onsonant?")
        reqs = True
        if choice.lower() == "v":
            let = letter_adder(vowels)
            print(let)
            letters.append(let)
        elif choice.lower() == "c":
            let = letter_adder(consonants)
            print(let)
            letters.append(let)
        else:
            print("Choose a vowel by typing V, or a consonant by typing C.")

        letters_string = "".join(letters)
        v,c = letter_classifier(letters_string)

        while c == 6 and len(letters)<9:
            print("Maximum number of consonants reached. Filling with vowels.")
            let = letter_adder(vowels)
            print(let)
            letters.append(let)
        while v == 5 and len(letters)<9:
            print("Maximum number of vowels reached. Filling with consonants.")
            let = letter_adder(consonants)
            print(let)
            letters.append(let) 
    return letters

# Word input

In [5]:
#word = input("What is your word?")

# Word List

In [6]:
import pandas as pd

In [7]:
df = pd.read_excel("Words.xlsx", engine='openpyxl')

# Word Checker

In [8]:
def word_check(st):
    #convert string to set in order to match alphagram
    st = st.upper()
    word = "".join(sorted(st))
    ref = df[["Alphagram","Word(s)"]]
    #does the alphagram match?
    alpha_match = (ref["Alphagram"]==word).any()
    if alpha_match:
        #sometimes there are multiple words that match the alphagram, so need to make sure that the word was spelled correctly
        ref_word = ref[(ref["Alphagram"]==word)]["Word(s)"].values[0]
        if "/" in ref_word:
            ref_words = ref_word.split("/")
            if st in ref_words:
                print(st + " is an acceptable word.")
                return True
            else:
                print(st + " is not an acceptable word.")
                return False
        elif st == ref_word:
            print(st + " is an acceptable word.")
            return True
        else:
            print(st + " is not an acceptable word.")
            return False
    else:
        print(st + " is not an acceptable word.")
        return False

In [9]:
def letter_check(st,letter_list):
    #need to check that the user input string uses only letters from the letter list and doesn't go over the limit
    #can't use sets because repeated letters are OK
    if len(st) > 9:
        print("Too many letters in "+ st)
        return False
    word_list = list(st)
    temp_letters = list(letter_list) #had to do this step or else the letter list got overwritten
    while (len(word_list))>0:
        if word_list[0] in temp_letters:
            temp_letters.remove(word_list[0])
            word_list.remove(word_list[0])
        else:
            print("Word is not valid. "+word_list[0].upper() +" is not in the list of acceptable letters")
            return False
            break
    if len(word_list)==0:
        return True
    else:
        print(st + "does not use the correct letters")
        return False

In [10]:
#letter_check(word,letters) and word_check(word)

# Find the best word

In [11]:
word_key = list(df["Single Word"].dropna())

In [12]:
def best_word(letter_list):
    matches = []
    for w in word_key:
        word_alpha = sorted(list(w))
        alpha = sorted([x.upper() for x in letter_list])
        while len(word_alpha) > 0:
            if word_alpha[0] in alpha:
                alpha.remove(word_alpha[0])
                word_alpha.remove(word_alpha[0])
            else:
                break
        if len(word_alpha) == 0:
            matches.append(w)
        else:
            continue
    best_df = df[df["Single Word"].isin(matches)]
    best_length = best_df["Length"].max()
    best = best_df[best_df["Length"]==best_length].sort_values(by="Weighted Usefulness Score",ascending=False)["Single Word"].reset_index(drop=True)[0]
    return matches,best

In [13]:
#dictionary lookup
from PyDictionary import PyDictionary
dictionary=PyDictionary()

def best_lookup(word):
    meaning = dictionary.meaning(word)
    return meaning

# all together

In [14]:
import time

In [15]:
letters = pick_letters()
print("".join(letters).upper())
#time.sleep(30)
word = input("What is your word?")
if letter_check(word,letters) and word_check(word):
    possible, bestword = best_word(letters)
    print("Possible words were: ")
    print(possible)
    print("The best word was "+ bestword)
    print(best_lookup(bestword))

(V)owel or (C)onsonant?v
{'a': 15, 'e': 21, 'i': 13, 'o': 12, 'u': 5}
o
(V)owel or (C)onsonant?v
{'a': 14, 'e': 21, 'i': 13, 'o': 12, 'u': 5}
a
(V)owel or (C)onsonant?c
{'b': 2, 'c': 3, 'd': 6, 'f': 2, 'g': 2, 'h': 2, 'j': 1, 'k': 1, 'l': 5, 'm': 4, 'n': 8, 'p': 4, 'q': 1, 'r': 9, 's': 9, 't': 9, 'v': 1, 'w': 1, 'x': 1, 'y': 1, 'z': 1}
g
(V)owel or (C)onsonant?c
{'b': 2, 'c': 3, 'd': 6, 'f': 2, 'g': 2, 'h': 2, 'j': 1, 'k': 1, 'l': 5, 'm': 4, 'n': 8, 'p': 3, 'q': 1, 'r': 9, 's': 9, 't': 9, 'v': 1, 'w': 1, 'x': 1, 'y': 1, 'z': 1}
p
(V)owel or (C)onsonant?c
{'b': 2, 'c': 3, 'd': 6, 'f': 2, 'g': 2, 'h': 2, 'j': 1, 'k': 1, 'l': 5, 'm': 4, 'n': 8, 'p': 2, 'q': 1, 'r': 9, 's': 9, 't': 9, 'v': 1, 'w': 1, 'x': 1, 'y': 1, 'z': 1}
p
(V)owel or (C)onsonant?c
{'b': 2, 'c': 3, 'd': 6, 'f': 2, 'g': 2, 'h': 2, 'j': 0, 'k': 1, 'l': 5, 'm': 4, 'n': 8, 'p': 2, 'q': 1, 'r': 9, 's': 9, 't': 9, 'v': 1, 'w': 1, 'x': 1, 'y': 1, 'z': 1}
j
(V)owel or (C)onsonant?v
{'a': 14, 'e': 20, 'i': 13, 'o': 12, 'u': 5}
e


In [ ]:
#TODO: update to use oxford dictionary for the lookup